In [440]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark import SparkContext, SparkConf

spark = SparkSession.builder.appName("Fake-job-prediction").getOrCreate()

In [441]:
%%html
<style>
div.output_area pre {
    white-space: pre;
}
</style>

In [442]:
df = spark.read.format("csv").option("header", "true").option("escape","\"").option("quote","\"").load("fake_job_postings.csv", sep=",")

df.show(20)

+------+--------------------+--------------------+----------+-------------+--------------------+--------------------+--------------------+--------------------+-------------+----------------+-------------+---------------+-------------------+--------------------+--------------------+--------------------+----------+
|job_id|               title|            location|department| salary_range|     company_profile|         description|        requirements|            benefits|telecommuting|has_company_logo|has_questions|employment_type|required_experience|  required_education|            industry|            function|fraudulent|
+------+--------------------+--------------------+----------+-------------+--------------------+--------------------+--------------------+--------------------+-------------+----------------+-------------+---------------+-------------------+--------------------+--------------------+--------------------+----------+
|     1|    Marketing Intern|    US, NY, New York| Mark

In [443]:
df.dtypes

[('job_id', 'string'),
 ('title', 'string'),
 ('location', 'string'),
 ('department', 'string'),
 ('salary_range', 'string'),
 ('company_profile', 'string'),
 ('description', 'string'),
 ('requirements', 'string'),
 ('benefits', 'string'),
 ('telecommuting', 'string'),
 ('has_company_logo', 'string'),
 ('has_questions', 'string'),
 ('employment_type', 'string'),
 ('required_experience', 'string'),
 ('required_education', 'string'),
 ('industry', 'string'),
 ('function', 'string'),
 ('fraudulent', 'string')]

In [444]:
from pyspark.sql.types import StringType

def get_country(location):
    if location is None:
        return None
    else:
        location_list = location.strip().split(sep = ",")
        return location_list[0].lower()

def get_state(location):
    if location is None:
        return None
    else:
        location_list = location.strip().split(sep = ",")
        if(len(location_list)>1 and not location_list[1].isspace()):
            return location_list[1].lower()
        else: 
            return None

udf_country = F.udf(get_country, StringType())
udf_state = F.udf(get_state, StringType())


In [445]:
df = df.withColumn("country", udf_country(F.col('location')))
df = df.withColumn("state", udf_state(F.col('location')))

#Due to the poor form of the location column, country and state are the only useful columns we can get.
df = df.drop(F.col('location'))

df.show(5)

+------+--------------------+----------+------------+--------------------+--------------------+--------------------+--------------------+-------------+----------------+-------------+---------------+-------------------+------------------+--------------------+--------------------+----------+-------+-----+
|job_id|               title|department|salary_range|     company_profile|         description|        requirements|            benefits|telecommuting|has_company_logo|has_questions|employment_type|required_experience|required_education|            industry|            function|fraudulent|country|state|
+------+--------------------+----------+------------+--------------------+--------------------+--------------------+--------------------+-------------+----------------+-------------+---------------+-------------------+------------------+--------------------+--------------------+----------+-------+-----+
|     1|    Marketing Intern| Marketing|        null|We're Food52, and...|Food52, a f

## Since the salary_range column may vary because of different currencies, we get the currency for each country

In [446]:
df.select("country").distinct().count()

91

In [447]:
from bs4 import BeautifulSoup
import requests

URL = "https://unece.org/fileadmin/DAM/cefact/recommendations/bkup_htm/cocucod.htm"
r = requests.get(URL)
soup = BeautifulSoup(r.content)

In [448]:
country_codes = []
currency_codes = []

info = soup.find_all("font", attrs={'color':'#000000','size':'1','face':'Verdana'})

country_codes = [info[x].text.lower() for x in range(1526) if len(info[x].text)==2]

currency_codes = [info[x].text.lower() for x in range(1526) 
                  if len(info[x].text)==3 
                  and not info[x].text.isnumeric() 
                  and info[x].text.isupper()]

#There are three countries with no official currency in this table, so we have to drop them.
remove_list = ['gs','ps','aq']

country_codes = [x for x in country_codes if x not in remove_list]

CC = list(zip(country_codes,currency_codes))

len(CC)

250

In [449]:
def get_currencycode(country):
    idx = 0
    for i in range(len(CC)):
        if CC[i][0]==country:
            return CC[idx][1]
        idx+=1
    return None       
    

udf_currency = F.udf(get_currencycode, StringType())


df = df.withColumn('salary_currency', udf_currency(F.col('country')))

In [450]:
from pyspark.sql.types import IntegerType

df = df.withColumn('salary_range', F.lower(F.col('salary_range')))

def get_minRange(salary):
    if salary is None:
        return None
    salary_range = salary.strip().split(sep = "-")
    try: 
        r = int(salary_range[0])
        return r 
    except ValueError: 
        return None

def get_maxRange(salary):
    if salary is None:
        return None
    salary_range = salary.strip().split(sep = "-")
    try:
        if(len(salary_range)==1):
            r = int(salary_range[0])    
        else:
            r = int(salary_range[1])
        return r
    except ValueError: 
        return None

udf_minRange = F.udf(get_minRange, IntegerType())
udf_maxRange = F.udf(get_maxRange, IntegerType())

df = df.withColumn('salary_min', udf_minRange(F.col('salary_range')))
df = df.withColumn('salary_max', udf_maxRange(F.col('salary_range')))

df = df.drop(F.col('salary_range'))

df.show(5)

+------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+-------------+----------------+-------------+---------------+-------------------+------------------+--------------------+--------------------+----------+-------+-----+---------------+----------+----------+
|job_id|               title|department|     company_profile|         description|        requirements|            benefits|telecommuting|has_company_logo|has_questions|employment_type|required_experience|required_education|            industry|            function|fraudulent|country|state|salary_currency|salary_min|salary_max|
+------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+-------------+----------------+-------------+---------------+-------------------+------------------+--------------------+--------------------+----------+-------+-----+---------------+----------+----------+
|     1|  

In [451]:
URL_CURRENCIES = 'https://api.exchangerate-api.com/v4/latest/USD'

curr = requests.get(URL_CURRENCIES).json()
curr = curr['rates']

curr =  {k.lower(): v for k, v in curr.items()}

In [452]:
def get_usd(number, currency):
    if number is None or currency is None:
        return None
    else:
        return int(number/curr[currency])

udf_usd = F.udf(get_usd, IntegerType())

df = df.withColumn('salary_min', udf_usd(F.col('salary_min'),F.col('salary_currency')))
df = df.withColumn('salary_max', udf_usd(F.col('salary_max'),F.col('salary_currency')))

In [453]:
df.show()

+------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+-------------+----------------+-------------+---------------+-------------------+--------------------+--------------------+--------------------+----------+-------+-----+---------------+----------+----------+
|job_id|               title|department|     company_profile|         description|        requirements|            benefits|telecommuting|has_company_logo|has_questions|employment_type|required_experience|  required_education|            industry|            function|fraudulent|country|state|salary_currency|salary_min|salary_max|
+------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+-------------+----------------+-------------+---------------+-------------------+--------------------+--------------------+--------------------+----------+-------+-----+---------------+----------+----------+
|   